In [78]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [79]:
import cuml 
import cupy as cp
import cudf as cd
import seaborn as sns

In [80]:
df = cd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")

In [81]:
df.head(10)

In [82]:
df.info()

In [83]:
df.shape

In [84]:
null_column = []
for col in df.columns:
    if df[col].isnull().sum()>=1:
        null_column.append(col)

In [85]:
df_train = df.drop(null_column,axis = 1)

In [86]:
df_train.head(4)

In [87]:
df_train.info()

In [88]:
final_df = df_train[['OverallQual', 'GrLivArea', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF', 'FullBath']]
final_df.head()

In [89]:
X = final_df
Y = df_train['SalePrice']

In [90]:
X

In [91]:
from cuml.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [92]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_scaled.as_matrix(),Y,test_size = 0.25,random_state = 42)

In [93]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']

In [94]:
from cuml import LinearRegression
err = 9999999999
final_lr = None
for alg in algorithm:
    print("Algorithm:")
    print(alg)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = alg)
    reg = lr.fit(X_train,Y_train)
    preds = lr.predict(X_test)
    Y_test=Y_test.astype(cp.float64)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
    if err>=cuml.metrics.regression.mean_squared_error(Y_test,preds):
        err = cuml.metrics.regression.mean_squared_error(Y_test,preds)
        final_lr = lr
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y_test,preds))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

In [95]:
test = cd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test.head()
test_id = test[['Id']]

In [96]:
test = test[['OverallQual', 'GrLivArea', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF', 'FullBath']]
test=test.fillna(0)
test.head()

In [97]:
pred = final_lr.predict(test)

In [98]:
type(test_id)
test_id = test_id.to_pandas()

In [99]:
type(pred)

In [100]:
import pandas as pd
output = pd.DataFrame()


In [101]:
output["Id"] = test_id

In [102]:
output["SalePrice"] = pd.DataFrame(pred)

In [103]:

output.to_csv('submission.csv', index=False)